In [1]:
import tkinter as tk
import math, random, time
from datetime import datetime

class FuturisticClock:
    def __init__(self, root):
        self.root = root
        self.root.title("Galaxy HUD Clock")
        self.root.attributes("-fullscreen", True)
        self.width = root.winfo_screenwidth()
        self.height = root.winfo_screenheight()

        self.canvas = tk.Canvas(root, bg="black", highlightthickness=0)
        self.canvas.pack(fill="both", expand=True)

        # Clock parameters
        self.cx, self.cy = self.width // 2, self.height // 2
        self.clock_radius = min(self.width, self.height) * 0.25

        # Starfield setup
        self.stars = []
        for _ in range(200):
            x = random.randint(0, self.width)
            y = random.randint(0, self.height)
            size = random.choice([1, 2])
            speed = random.uniform(0.2, 1.0)
            self.stars.append([x, y, size, speed])

        # Animation vars
        self.ring_angle = 0
        self.pulse_phase = 0

        self.update_clock()

    def draw_starfield(self):
        for star in self.stars:
            x, y, size, speed = star
            brightness = random.randint(180, 255)
            color = f"#{brightness:02x}{brightness:02x}{brightness:02x}"
            self.canvas.create_oval(x-size, y-size, x+size, y+size, fill=color, outline="")
            # Move star downward slowly (wrap around)
            star[1] += speed
            if star[1] > self.height:
                star[0] = random.randint(0, self.width)
                star[1] = 0

    def draw_rotating_rings(self):
        ring_colors = ["#00ffaa", "#00ccff"]
        self.ring_angle += 0.5
        for i, radius in enumerate([self.clock_radius*1.1, self.clock_radius*1.25]):
            for deg in range(0, 360, 15):
                angle = math.radians(deg + self.ring_angle*(1 if i%2==0 else -1))
                x = self.cx + radius * math.cos(angle)
                y = self.cy + radius * math.sin(angle)
                self.canvas.create_oval(x-3, y-3, x+3, y+3, fill=ring_colors[i], outline="")

    def draw_clock_face(self):
        # Hour markers
        for h in range(12):
            ang = math.radians(h*30 - 90)
            x1 = self.cx + (self.clock_radius-20)*math.cos(ang)
            y1 = self.cy + (self.clock_radius-20)*math.sin(ang)
            x2 = self.cx + self.clock_radius*math.cos(ang)
            y2 = self.cy + self.clock_radius*math.sin(ang)
            self.canvas.create_line(x1, y1, x2, y2, fill="#00ffaa", width=3)

    def draw_hands(self, now):
        # Time calculations
        hour = now.hour % 12 + now.minute / 60.0
        minute = now.minute + now.second / 60.0
        second = now.second + now.microsecond / 1_000_000.0

        # Angles
        hour_ang = math.radians(hour * 30 - 90)
        min_ang  = math.radians(minute * 6 - 90)
        sec_ang  = math.radians(second * 6 - 90)

        # Neon glow hands (layered)
        self.draw_glow_line(self.cx, self.cy, self.cx + self.clock_radius*0.5*math.cos(hour_ang),
                            self.cy + self.clock_radius*0.5*math.sin(hour_ang), "#00e6ff", 10)
        self.draw_glow_line(self.cx, self.cy, self.cx + self.clock_radius*0.75*math.cos(min_ang),
                            self.cy + self.clock_radius*0.75*math.sin(min_ang), "#00ffaa", 6)
        self.draw_glow_line(self.cx, self.cy, self.cx + self.clock_radius*0.9*math.cos(sec_ang),
                            self.cy + self.clock_radius*0.9*math.sin(sec_ang), "#ff3366", 2)

    def draw_glow_line(self, x1, y1, x2, y2, color, width):
        # Multiple layers for glow effect
        for w in range(width+4, width, -1):
            self.canvas.create_line(x1, y1, x2, y2, fill=color, width=w, stipple="gray25")
        self.canvas.create_line(x1, y1, x2, y2, fill=color, width=width)

    def draw_center_pulse(self):
        self.pulse_phase += 0.1
        size = 15 + 5*math.sin(self.pulse_phase)
        self.canvas.create_oval(self.cx-size, self.cy-size, self.cx+size, self.cy+size,
                                fill="#ffffff", outline="")

    def draw_digital_clock(self, now):
        digital_time = now.strftime("%H:%M:%S")
        date_str = now.strftime("%A, %d %B %Y")
        self.canvas.create_text(self.cx, self.cy - self.clock_radius - 80,
                                text=digital_time, font=("Orbitron", 60, "bold"),
                                fill="#00ffcc")
        self.canvas.create_text(self.cx, self.cy - self.clock_radius - 40,
                                text=date_str, font=("Orbitron", 20),
                                fill="#99ffcc")

    def update_clock(self):
        self.canvas.delete("all")

        now = datetime.now()

        self.draw_starfield()
        self.draw_rotating_rings()
        self.draw_clock_face()
        self.draw_hands(now)
        self.draw_center_pulse()
        self.draw_digital_clock(now)

        self.root.after(50, self.update_clock)

if __name__ == "__main__":
    root = tk.Tk()
    FuturisticClock(root)
    root.mainloop()